# Задача 1 
## Консолидация данных по семьям

In [44]:
import pandas as pd
import numpy as np
import datetime
import os

In [45]:
raw_data = pd.read_excel('выгрузка анкет.xlsx', dtype=str)  # предварительно убрать все объединенные ячейки (разъединить)
questionaries = raw_data.copy() # скопируем для задачи 1





In [46]:
# оставим только необходимые данные по членам семьи

not_necessary_columns = ['Заполняли Вы ранее подобную Анкету в электронном или бумажном виде?',
                         'Хотели бы Вы получать дополнительную информацию об услугах и специальных акциях по e-mail?',
                         'В нашем центре есть Центр Амбулаторной Хирургии, который проводит операции любой сложности',
                         'В нашем центре открылось новое направление «Косметология» и мы работаем с новым современным лазером',
                         'В нашем центре есть служба «Медицинское такси XXI век», как для клиентов в поездке по городу, так и специальные автомобили для пациентов с ограниченными возможностями к передвижению',
                         'Отметьте, пожалуйста, по шкале от 1 до 10, порекомендовали бы Вы Медицинский центр «XXI век» друзьям и близким? (1 – точно НЕ порекомендовал(а), 10 – точно порекомендовал(а) бы)',
                         'Где вы еще лечитесь?', 'Другие медцентры', 'Как вы добираетесь до клиники?', 'Как обычно записываетесь к врачу?',
                         'Что могло бы сделать посещение более приятным для вас?', 'Какие услуги вы хотели бы видеть в нашем центре?',
                         'Если вы хотите сказать СПАСИБО кому-то из наших сотрудников, то можете сделать это тут', 'Разрешаю публикацию благодарности на интернет-ресурсах Медицинского Центра "XXI Век" * ​* При публикации благодарности ваши личные данные будут скрыты.']

questionaries = questionaries[[i for i in questionaries.columns if i not in not_necessary_columns]]


In [47]:
# не будем брать пустые строки

questionaries = questionaries[~questionaries.isnull().all(axis=1)]

In [48]:
def fill(lst):
    '''Пронумеровать все строки на их принадлежность к предыдущей
    Сохранить в новый столбец Номер анкеты'''
    arr = []
    cnt = 1
    for i, j in enumerate(lst):
        if i == 0:
            arr.append(1)
            continue
        if j is np.NaN:
            arr.append(cnt)
        else:
            cnt += 1
            arr.append(cnt)
    return arr

questionaries['Номер анкеты'] = fill(questionaries['Изменено'].tolist())

questionaries = questionaries[['Номер анкеты'] + [i for i in questionaries.columns if i != 'Номер анкеты']] # столбец в начало поставили


In [49]:
# На первоначальном этапе предлагаю проверять как заполнены столбцы в таблице, как их лучше соединить
# это, думаю, для задачи номер 2


# temp_data  = questionaries[questionaries['Изменено'].isnull()]

# for column in temp_data:
#     print(temp_data[column].isnull().value_counts())
#     print('\n\n')

In [50]:
# ниже циклом проходимся построчно по questionaries и создаем новый датафрейм с необходимой информацией

final_data = pd.DataFrame({'Номер анкеты': [], 'Изменено': [], 'Глава семьи': [], 'ФИО': [], 'Дата рождения': [], 'Телефон': [], 'E-mail': [], 'Кем приходится': [], 'Обслуживается у нас': [], 'Адрес проживания': []})


for i in questionaries.index:
    number = questionaries['Номер анкеты'][i]
    date = questionaries['Изменено'][i]
    head = questionaries['Ваши Ф.И.О. (взрослого члена семьи, который заполняет Анкету)'][i]
    fios = [questionaries[j][i] for j in questionaries.columns if 'Ф.И.О.' in j]
    birthday = [questionaries[j][i] for j in questionaries.columns if 'Дата рождения' in j]
    phones = [questionaries[j][i] for j in questionaries.columns if 'Телефон' in j]
    mails = [questionaries[j][i] for j in questionaries.columns if 'E-mail' in j]
    status =['Глава семьи'] + [questionaries[j][i] for j in questionaries.columns if 'Кем Вам приходится' in j]
    services = [questionaries[j][i] for j in questionaries.columns if 'служив' in j]
    adresses = [questionaries[j][i] for j in questionaries.columns if 'Адрес проживания' in j]
    len_fios = len(fios)
    temp_dct = pd.DataFrame({'Номер анкеты': [number] * len_fios, 
                'Изменено': [date] * len_fios, 
                'Глава семьи': [head] * len_fios, 
                'ФИО': fios, 
                'Дата рождения': birthday, 
                'Телефон': phones, 
                'E-mail': mails, 
                'Кем приходится': status,
                'Обслуживается у нас': services,
                'Адрес проживания': adresses})
    temp_dct.dropna(subset=['ФИО'], inplace=True)
    

    
    final_data = final_data.append(pd.DataFrame(temp_dct))
    


In [51]:
final_data['ФИО'] = final_data['ФИО'].str.strip().str.lower().str.replace('ё', 'е') # приводим все к единому знаменателю для поиска

## Сначала найдем по максимуму пациентов из базы, без оборота

In [52]:
pacients = pd.DataFrame()
cur_path = os.getcwd()
os.chdir(os.path.join(cur_path, 'pacients'))
for file in os.listdir('.'):
    if not pacients.shape[0]:
        pacients = pd.read_excel(file)
    else:
        pacients = pacients.append(pd.read_excel(file))
        
pacients = pacients.drop_duplicates(subset=['Номер карты']).reset_index(drop=True)        

pacients['Пациент'] = pacients['Пациент'].str.lower().str.replace('ё', 'е') # приводим все к единому знаменателю для поиска

In [53]:
def to_strf(x):
    if isinstance(x, datetime.datetime):
        return x.strftime('%d.%m.%Y')
    else:
        return x

pacients['Дата рождения'] = pacients['Дата рождения'].apply(to_strf)  # приводим все к единому знаменателю для поиска

In [54]:
final_data = final_data.merge(pacients, how='left', left_on=['ФИО', 'Дата рождения'], right_on=['Пациент', 'Дата рождения'])  # соединяем данные

In [55]:
final_data.drop('Пациент', axis=1, inplace=True)

final_data['Номер карты'] = final_data['Номер карты'].fillna('Нет данных')

In [56]:
os.chdir(cur_path)
final_data.to_excel('Сводные_данные_по_анкетам.xlsx', index=False)

In [57]:
pacients['Ном']

,Пациент,Дата рождения,Номер карты
0,,-,990313477
1,,08.12.1968,990375114
2,калеченкова наталия анатольевна,12.11.1973,990475670
3,- аноним,18.11.1986,100184866
4,- екатерина,05.12.1993,120583506
...,...,...,...
1321569,слепцова арина евгеньевна,08.05.2003,80271
1321570,слепцова анна филипповна,01.06.1976,140053941
1321571,слепцова анна витальевна,16.09.1973,100097463
1321572,слепцова анна вадимовна,16.04.1988,120551565
